In [10]:
from cval import CVal

config_file = "../config.toml"
env_file = "../.env"

cval = CVal.init(
    config_file=config_file,
    env_file=env_file
)

[06/25/24 15:15:45] INFO     Ingestion engine initialized.                                          ]8;id=165435;file:///home/simisimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=771968;file:///home/simisimon/GitHub/cval/src/ingestion.py#42\42]8;;\

                    INFO     Retrieval engine initialized.                                          ]8;id=493070;file:///home/simisimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=657094;file:///home/simisimon/GitHub/cval/src/retrieval.py#29\29]8;;\

                    INFO     Ingestion engine initialized.                                          ]8;id=657258;file:///home/simisimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=778978;file:///home/simisimon/GitHub/cval/src/ingestion.py#42\42]8;;\

                    INFO     CVal initialized.                                                          ]8;id=465186;file:///home/simisimon/GitHub/cval/src/cval.py\cval.py]8;;\:]8;id=546055;file:///home/simisimon/GitHub/cval/src/cval.py#101\101]8;;\

In [11]:
import pandas as pd

eval_data_file = "../data/eval/dependencies_without_rules.csv"

df = pd.read_csv(eval_data_file, sep=";")

df_sample = df[:5]

df_sample

,link_str,project,option_name,option_value,option_file,option_type,option_technology,dependent_option_name,dependent_option_value,dependent_option_file,dependent_option_type,dependent_option_technology,pre_rating,rating,Notes
0,mall::::mall-admin/src/main/resources/applicat...,mall,spring.datasource.druid.stat-view-servlet.logi...,druid,mall-admin/src/main/resources/application-dev.yml,ConfigType.PASSWORD,spring,spring.datasource.druid.stat-view-servlet.logi...,druid,mall-search/src/main/resources/application-pro...,ConfigType.PASSWORD,spring,False,borderline,"false if each service has its own service, tru..."
1,mall::::mall-mbg/pom.xml::::project::::groupId...,mall,project.groupId,com.macro.mall,mall-mbg/pom.xml,ConfigType.NAME,maven,project.parent_mall.groupId,com.macro.mall,mall-mbg/pom.xml,ConfigType.NAME,maven,True,True,NaN
2,apollo::::apollo-biz/src/test/resources/applic...,apollo,spring.h2.console.enabled,true,apollo-biz/src/test/resources/application.prop...,ConfigType.BOOLEAN,spring,project.build.plugins.plugin_maven-jar-plugin....,true,pom.xml,ConfigType.IP_ADDRESS,maven,False,False,NaN
3,apollo::::apollo-assembly/pom.xml::::project::...,apollo,project.dependencies.dependency_apollo-portal....,apollo-portal,apollo-assembly/pom.xml,ConfigType.NAME,maven,COPY.src,apollo-portal,apollo-portal/src/main/docker/Dockerfile,ConfigType.PATH,docker,False,False,NaN
4,apollo::::apollo-assembly/pom.xml::::project::...,apollo,project.parent_apollo.relativePath,../pom.xml,apollo-assembly/pom.xml,ConfigType.PATH,maven,project.parent_apollo.relativePath,../pom.xml,apollo-portal/pom.xml,ConfigType.PATH,maven,NaN,True,NaN


In [12]:
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase
from dotenv import dotenv_values
import os

config = dotenv_values(dotenv_path="../.env")
os.environ['OPENAI_API_KEY'] = config["OPENAI_KEY"]


from deepeval.models.base_model import DeepEvalBaseLLM
from llama_index.llms.ollama import Ollama

model = Ollama(model="llama3:8b", temperature=0.0)

class LlamaModel(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()
        return model.complete(prompt).text

    async def a_generate(self, prompt: str) -> str:
        model = self.load_model()
        res = await model.acomplete(prompt)
        return res.text

    def get_model_name(self):
        return "Custom Llama3 model"


answer_relevancy_metric = AnswerRelevancyMetric(
    threshold=0.7,
    model="gpt-4",
    include_reason=True
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.7,
    model="gpt-4",
    include_reason=True
)

context_relevancy_metric = ContextualRelevancyMetric(
    threshold=0.7,
    model="gpt-4",
    include_reason=True
)

In [13]:
from data import Dependency
import mlflow
import datetime
import json

index_name = "all"
exp_name = "dependencies_without_rules"
date_time = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

mlflow.set_experiment(experiment_name=exp_name)

with mlflow.start_run(run_name=f"{exp_name}_{date_time}"):

    output = []

    for x in df_sample.to_dict("records"):
        dependency = Dependency(
            project=x["project"],
            option_name=x["option_name"],
            option_value=x["option_value"],
            option_type=x["option_type"].split(".")[-1],
            option_file=x["option_file"],
            option_technology=x["option_technology"],
            dependent_option_name=x["dependent_option_name"],
            dependent_option_value=x["dependent_option_value"],
            dependent_option_type=x["dependent_option_type"].split(".")[-1],
            dependent_option_file=x["dependent_option_file"],
            dependent_option_technology=x["dependent_option_technology"]
        )

        response = cval.query(
            dependency=dependency,
            index_name=index_name
        )

        output.append(response)

    inputs = [x.input for x in output]
    results = [x.response for x in output]
    mlflow.log_table(data={"inputs": inputs, "outputs": results}, artifact_file="results.json")

    

[06/25/24 15:15:46] INFO     Select index: github.                                                  ]8;id=342464;file:///home/simisimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=915094;file:///home/simisimon/GitHub/cval/src/retrieval.py#47\47]8;;\

[06/25/24 15:15:48] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=582166;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=381174;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[06/25/24 15:15:49] INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=552980;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=320480;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

[06/25/24 15:15:50] INFO     Len of retrieved nodes of query 0: 5                                  ]8;id=954873;file:///home/simisimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=182344;file:///home/simisimon/GitHub/cval/src/retrieval.py#174\174]8;;\

                    INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=927829;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=660503;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

...

                    INFO     Len of retrieved nodes after reranking: 3                             ]8;id=771852;file:///home/simisimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=386683;file:///home/simisimon/GitHub/cval/src/retrieval.py#117\117]8;;\

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.19it/s]


[06/25/24 15:19:03] INFO     Len of retrieved nodes after reranking: 3                             ]8;id=908719;file:///home/simisimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=765699;file:///home/simisimon/GitHub/cval/src/retrieval.py#117\117]8;;\

                    INFO     GPT (gpt-4o-2024-05-13) generator initialized.                         ]8;id=543554;file:///home/simisimon/GitHub/cval/src/generator.py\generator.py]8;;\:]8;id=282044;file:///home/simisimon/GitHub/cval/src/generator.py#46\46]8;;\

[06/25/24 15:19:09] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=198816;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=450217;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

In [14]:
faithfulness_scores = []
answer_relevancy_scores = []
context_relevancy_scores = []

false_positives = []
true_positives = []
false_negatives = []
true_negatives = []
accuracy = []

for response, baseline in zip(output, df_sample.to_dict("records")):


    test_case = LLMTestCase(
        input=response.input,
        actual_output=response.response,
        retrieval_context=[source_node.node.get_content() for source_node in response.source_nodes]
    )

    context_relevancy_metric.measure(test_case)
    faithfulness_metric.measure(test_case)
    answer_relevancy_metric.measure(test_case)

    context_relevancy_scores.append(context_relevancy_metric.score)
    faithfulness_scores.append(faithfulness_metric.score)
    answer_relevancy_scores.append(answer_relevancy_metric.score)       
            
    response_dict = json.loads(response.response)
        
    print("Context Relevancy: ", context_relevancy_scores, sum(context_relevancy_scores)/len(context_relevancy_scores))
    print("Answer Relevancy: ", answer_relevancy_scores, sum(answer_relevancy_scores)/len(answer_relevancy_scores))
    print("Faithfulness: ", faithfulness_scores, sum(faithfulness_scores)/len(faithfulness_scores))

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Event loop is already running. Applying nest_asyncio patch to allow async execution...

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[06/25/24 15:19:13] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=546561;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=146411;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=529577;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=299225;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

...

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[06/25/24 15:24:19] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=874850;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=556140;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[06/25/24 15:24:22] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=486314;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=894408;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Context Relevancy:  [0.0, 1.0, 0.0, 0.0, 0.6666666666666666] 0.3333333333333333
Answer Relevancy:  [1.0, 1.0, 1.0, 1.0, 1.0] 1.0
Faithfulness:  [1.0, 1.0, 1.0, 1.0, 1.0] 1.0


In [15]:
true_positives = []
true_negatives = []
false_positives = []
false_negatives = []
accuracy = []

for response, baseline in zip(output, df_sample.to_dict("records")):
    response_dict = json.loads(response.response)


    if baseline["rating"] in ("True", "Unsure") and response_dict["isDependency"]:
        accuracy.append(1)
        true_positives.append(1)
    if baseline["rating"] in ("False") and not response_dict["isDependency"]:
        accuracy.append(1)
        true_negatives.append(1)
    if baseline["rating"] in ("True", "Unsure") and not response_dict["isDependency"]:
        accuracy.append(0)
        false_negatives.append(1)
    if baseline["rating"] in ("False") and response_dict["isDependency"]:
        accuracy.append(0)
        false_positives.append(1)


#precision = sum(true_positives)/(sum(true_positives)+sum(false_positives))
#recall = sum(true_positives)/(sum(true_positives)+sum(false_negatives))
#f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy", sum(accuracy)/len(accuracy))
print("TP", sum(true_positives))
print("FP", sum(false_positives))
print("TN", sum(true_negatives))
print("FN", sum(false_negatives))
#print("Precision", precision)
#print("Recall", recall)
#print("F1 Score: ", f1_score)

Accuracy 0.75
TP 1
FP 0
TN 2
FN 1


In [16]:
#inputs = [x.input for x in output]
results = [x.response for x in output]
rating = df_sample["rating"]

df_results = pd.DataFrame().from_dict(data={"outputs": results, "isDependency": rating})
df_results.to_csv(f"../data/results/test_{index_name}.csv", index=False)


In [17]:
for x in output:
    print(x.input)
    print(x.response)
    break

You are a full-stack expert in validating intra-technology and cross-technology configuration dependencies.
You will be presented with configuration options found in the software project 'mall'.
Your task is to determine whether the given configuration options actually depend on each other based on value-equality.

A value-equality dependency is present if two configuration options must have identical values in order to function correctly.
Inconsistencies in these configuration values can lead to configuration errors. 
Importantly, configuration options may have equal values by accident, meaning that there is no actual dependency, but it just happens that they have equal values.
Information about both configuration options, including their descriptions and prior usages are stated below:
The meaning of each configuration is already listed in the code.

Next, open the StatViewServlet configuration, as follows.

    
    
     1
     2
     3
     4
     5
     6
     7
     8
     9
    